In [1]:
import pandas as pd

In [2]:
sp500  = pd.read_csv("https://datahub.io/core/s-and-p-500-companies/_r/-/data/constituents.csv")
sp500_symbols = set(sp500["Symbol"])

In [3]:
# Nasdaq-100 table – first wikitable on that page
nasdaq100 = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]   # table #3 is constituents
nasdaq100_symbols = set(nasdaq100["Ticker"])

In [4]:
from pathlib import Path
import json, gzip, pickle
import pandas as pd

In [5]:
def save_tickers(tickers: set[str], name: str, out_dir="ticker_lists"):
    """
    Save a set of tickers to disk in TXT, CSV, JSON and compressed Pickle.
    
    Parameters
    ----------
    tickers : set[str]
        Unique tickers (case-sensitive).
    name : str
        Base filename (e.g. "sp500", "nasdaq100").
    out_dir : str | Path
        Folder where files will be written.
    """
    out = Path(out_dir)
    out.mkdir(exist_ok=True)

    # 1 — Plain-text, one ticker per line
    (out / f"{name}.txt").write_text("\n".join(sorted(tickers)))

    # 2 — CSV (no header, no index)
    pd.Series(sorted(tickers)).to_csv(out / f"{name}.csv",
                                      index=False, header=False)

    # 3 — JSON list
    json.dump(sorted(tickers), open(out / f"{name}.json", "w"))

    # 4 — Compressed pickle (fastest for Python re-load)
    with gzip.open(out / f"{name}.pkl.gz", "wb") as f:
        pickle.dump(tickers, f, protocol=4)

    print(f"✔ saved {len(tickers):,} tickers to {out.resolve()}")

In [6]:
save_tickers(sp500_symbols,  "sp500")
save_tickers(nasdaq100_symbols, "nasdaq100")

✔ saved 503 tickers to /home/mengren/projects/Polygon.io-data-ingestion-pipeline-main/ticker_lists
✔ saved 101 tickers to /home/mengren/projects/Polygon.io-data-ingestion-pipeline-main/ticker_lists


In [7]:
len(sp500_symbols), len(nasdaq100_symbols), len(sp500_symbols | nasdaq100_symbols)

(503, 101, 517)

In [8]:
save_tickers(sp500_symbols | nasdaq100_symbols, "spx_ndx_combined")

✔ saved 517 tickers to /home/mengren/projects/Polygon.io-data-ingestion-pipeline-main/ticker_lists
